In [8]:
import tensorflow as tf
import dill
import sys
sys.path.append("..")

from utils.helper import *

In [9]:
with open(f"../Models/AR.pkl", 'rb') as file:
    AR = dill.load(file)

with open(f"../Models/EN.pkl", 'rb') as file:
    EN = dill.load(file)

In [10]:
with open("../Models/encoder.json", "r") as file:
    encoder_json = file.read()

# Create an instance of your subclassed model
encoder = tf.keras.models.model_from_json(encoder_json, custom_objects={'Encoder': Encoder})

encoder(tf.zeros((64, 1)),tf.zeros((64, 1024)))
# Load the model's weights
encoder.load_weights("../Models/encoder_en2ar.h5")

In [11]:
with open("../Models/decoder.json", "r") as file:
    decoder_json = file.read()

# Create an instance of your subclassed model
decoder = tf.keras.models.model_from_json(decoder_json, custom_objects={'Decoder': Decoder})

decoder(tf.zeros((64, 1)),tf.zeros((64, 1024)),tf.zeros((64, 1, 1024)))
# Load the model's weights
decoder.load_weights("../Models/decoder_en2ar.h5")

In [12]:
def evaluate(sentence, encoder, decoder):
    # attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence,"en")

    # inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = EN.texts_to_sequences([sentence])[0]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=EN.maxlen, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, 1024))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([AR.word2idx['<start>']], 0)

    for t in range(AR.maxlen):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.random.categorical(tf.exp(predictions), num_samples=1)[0][0].numpy()

        result += AR.idx2word[predicted_id] + ' '

        if AR.idx2word[predicted_id] == '<end>':
            return result, sentence 
        
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence 

In [13]:
evaluate("hello people", encoder, decoder)

('اهلا لمزيد من كندا . <end> ', '<start> hello people <end>')